## Project proposal
### Importing the dataset

In [2]:
install.packages("cowplot")
install.packages("kknn")
install.packages("GGally")
install.packages('tidyverse')
library(kknn)
library(GGally)


The downloaded binary packages are in
	/var/folders/fb/5nf2l6vx4tj8839ndlppjshw0000gn/T//Rtmp40wrHo/downloaded_packages

The downloaded binary packages are in
	/var/folders/fb/5nf2l6vx4tj8839ndlppjshw0000gn/T//Rtmp40wrHo/downloaded_packages

The downloaded binary packages are in
	/var/folders/fb/5nf2l6vx4tj8839ndlppjshw0000gn/T//Rtmp40wrHo/downloaded_packages

The downloaded binary packages are in
	/var/folders/fb/5nf2l6vx4tj8839ndlppjshw0000gn/T//Rtmp40wrHo/downloaded_packages


Loading required package: ggplot2

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



In [3]:
#import libraries
install.packages("themis")
library(tidyverse)
library(repr)
options(repr.matrix.max.rows = 16)
library(readxl)
library(ggplot2)
library(cowplot)


The downloaded binary packages are in
	/var/folders/fb/5nf2l6vx4tj8839ndlppjshw0000gn/T//Rtmp40wrHo/downloaded_packages


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




## Dataset background
- The dataset information can be found at: https://archive.ics.uci.edu/dataset/320/student+performance
## Possible questions from the dataset
- Proportion of students getting high to very high freetime after school in urban area in comparison of proportion of students getting high to very high freetime after school in rural area
- Mean final grade of students receiving extra support from school in comparison to students getting no support from school
- The mean number of absences for students in urban area in comparison to students in rural area. 
- Mean number of past class failures for students with family support in comparison to students with no family support. 
- mean final grade of students with a romantic relationship in comparison to the mean final grade of students with no romantic relationship.

## Final question for the dataset:
- Variables that we are interested in: freetime after school that are not related to schoolwork, and address indicating the student home is either in the urban area or rural area
- With this dataset, we would like to explore: Proportion of students getting high to very high freetime after school in urban area in comparison of proportion of students getting high to very high freetime after school in rural area
- With this dataset, one location parameter that we will use is proportion of students getting high to very high free time after school. Another scale parameter is the standard deviation to see if the difference between two groups are by chance or not

## Import dataset

In [4]:
studenturl<- "https://archive.ics.uci.edu/static/public/320/student+performance.zip"
temp <- tempfile()
download.file(studenturl,temp)
studentzip <- unzip(temp, "student.zip")
studentfile <- unzip(studentzip,"student-por.csv")
student <- read_csv2(studentfile, col_names = TRUE)

student

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 649 Columns: 33
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (17): school, sex, address, famsize, Pstatus, Mjob, Fjob, reason, guardi...
dbl (16): age, Medu, Fedu, traveltime, studytime, failures, famrel, freetime...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GP,F,18,U,GT3,A,4,4,at_home,teacher,⋯,4,3,4,1,1,3,4,0,11,11
GP,F,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,2,9,11,11
GP,F,15,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,6,12,13,12
GP,F,15,U,GT3,T,4,2,health,services,⋯,3,2,2,1,1,5,0,14,14,14
GP,F,16,U,GT3,T,3,3,other,other,⋯,4,3,2,1,2,5,0,11,13,13
GP,M,16,U,LE3,T,4,3,services,other,⋯,5,4,2,1,2,5,6,12,12,13
GP,M,16,U,LE3,T,2,2,other,other,⋯,4,4,4,1,1,3,0,13,12,13
GP,F,17,U,GT3,A,4,4,other,teacher,⋯,4,1,4,1,1,1,2,10,13,13
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


## Clean and wrangle dataset
- For this dataset, we will select the variable of interest, along with filtering out any data that is invalid (i.e NA)
- After that we will filter out the freetime after school information to be 3 and above (indicating high to very high free time)

In [5]:
select_student <- student%>%
                  select(schoolsup,G3)
select_student
summary_student <- select_student%>%
                   group_by(schoolsup)%>%
                   summarize(mean_grade = mean(G3))

schoolsup,G3
<chr>,<dbl>
yes,11
no,11
yes,12
no,14
no,13
no,13
no,13
yes,13
⋮,⋮


## Plot the raw data
- We will plot the students freetime according to their address in a bargraph to see the proportion of students freetime in both category

In [54]:
bargraph <- summary_student %>%
            ggplot(aes(x = schoolsup,y=mean_grade)) +
            geom_boxplo(stat="identity")+
            xlab("Receive extra educational support from school") +
            ylab("Mean grade")

`summarise()` has grouped output by 'address'. You can override using the
`.groups` argument.


address,freetime,count
<chr>,<lgl>,<int>
R,TRUE,80
U,TRUE,166


In [56]:
prop_students <- mutate_student %>%
                 group_by(address,freetime)%>%
                 summarize(prop = n()/nrow(
prop_students

`summarise()` has grouped output by 'address'. You can override using the
`.groups` argument.


address,freetime,prop_urban,prop_rural
<chr>,<lgl>,<dbl>,<dbl>
R,TRUE,0.1769912,0.4060914
U,TRUE,0.3672566,0.8426396


In [ ]:
bargraph <- prop_students %>%
            ggplot()

### General plan
Question: Mean final grade of students receiving extra support from school in comparison to students getting no support from school
Preliminary: graph boxplot or sample distribution of the two group (receive school suppo/receive no support)
Method: 
- can the dataset be generalizable, random, representative to the population (population: all student)? Explain
- Explain why the boxplot or the sample distribution is not enough to estimate the true difference
- Write the hypothesis. What is null hypothesis, and what alternative hypothesis
- If the null hypothesis fail, what is the confidence interval for the mean difference (build using CLT, since bootstrapping with a large sample is computationally expensive)
- Pick a significance level (alpha)
- Impact (base on some paper that we find)
- Expect to find (alternative hypothesis)
- Future questions??????? (base on some paper)
